## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-09-10-39-50 +0000,bbc,More than 100 hurt as efforts to contain Marse...,https://www.bbc.com/news/articles/cp8mz44j6n6o
1,2025-07-09-10-38-57 +0000,bbc,Gaza truce talks reportedly stall despite seco...,https://www.bbc.com/news/articles/ce3newwl1zeo
2,2025-07-09-10-32-05 +0000,bbc,Ukraine suffers heaviest attack after Trump cr...,https://www.bbc.com/news/articles/c2k19q4j07zo
3,2025-07-09-10-13-19 +0000,bbc,Post Office compensation schemes 'cruel and ne...,https://www.bbc.com/news/articles/crk6r2615vzo
4,2025-07-09-10-06-54 +0000,bbc,US cuts visa validity for most Nigerian applic...,https://www.bbc.com/news/articles/cx23ywg9w82o


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2295/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
12,trump,13
138,uk,5
177,tariffs,5
60,mexico,4
59,new,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
25,2025-07-09-03-50-00 +0000,wsj,President Trump decided to delay the implement...,https://www.wsj.com/economy/trade/trump-tariff...,39
42,2025-07-08-21-38-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...,37
45,2025-07-08-20-58-00 +0000,wsj,Copper prices surged and the Dow industrials s...,https://www.wsj.com/economy/trade/trumps-tarif...,31
29,2025-07-09-01-00-00 +0000,wsj,Two Republicans named Kevin are vying to be th...,https://www.wsj.com/economy/central-banking/tr...,29
33,2025-07-08-23-44-53 +0000,bbc,'Lives are at stake:' Deadly Texas storms put ...,https://www.bbc.com/news/articles/cx23ye0xnnyo,28


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
25,39,2025-07-09-03-50-00 +0000,wsj,President Trump decided to delay the implement...,https://www.wsj.com/economy/trade/trump-tariff...
29,29,2025-07-09-01-00-00 +0000,wsj,Two Republicans named Kevin are vying to be th...,https://www.wsj.com/economy/central-banking/tr...
42,19,2025-07-08-21-38-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...
43,18,2025-07-08-21-31-55 +0000,bbc,King and Macron toast 'ever closer' UK-France ...,https://www.bbc.com/news/articles/cvg87y6d5j4o
8,17,2025-07-09-09-32-29 +0000,bbc,At least three dead in New Mexico flash flooding,https://www.bbc.com/news/articles/cx2jk51xv65o
33,15,2025-07-08-23-44-53 +0000,bbc,'Lives are at stake:' Deadly Texas storms put ...,https://www.bbc.com/news/articles/cx23ye0xnnyo
45,15,2025-07-08-20-58-00 +0000,wsj,Copper prices surged and the Dow industrials s...,https://www.wsj.com/economy/trade/trumps-tarif...
46,15,2025-07-08-19-52-56 +0000,bbc,Canadian Armed Forces members plotted 'anti-go...,https://www.bbc.com/news/articles/cly8d0jxm8po
6,12,2025-07-09-09-41-48 +0000,nyt,Philadelphia Public Workers Reach Deal to End ...,https://www.nytimes.com/2025/07/09/world/ameri...
53,12,2025-07-08-16-41-29 +0000,bbc,Imposter used AI to pose as Marco Rubio and co...,https://www.bbc.com/news/articles/crrqkyyjewno


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
